In [ ]:
print("---------preparing training data for knowledge base features")

In [1]:
import sys,json,re,string
from typing import Callable, Dict, List, Set, Tuple
from elasticsearch import Elasticsearch
import gensim.downloader as api
import gensim
import datetime
import csv
sys.path.insert(1, 'extract_features')
sys.path.insert(1, 'util')
from helper_function import preprocess
from extract_No1to5_features import extract_features_1to5
from extract_No11to12_features import extract_features_11to12
from extract_No13to15_features import TypeTaxonomy, extract_features_13to15
from extract_No16_feature import extract_features_16
from extract_No17to19_features import get_analyze,extract_features_17to19
from extract_No20to22_features import extract_features_20to22
from extract_No23to25_features_optimized import extract_features_23to25

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\junec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\junec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [10]:
# Reduce logging level.
import logging
logging.basicConfig(level=logging.ERROR)

logging.getLogger("elasticsearch").disabled = True
es= Elasticsearch(timeout=600)
es.info()

{'name': 'LAPTOP-ADBLIUPR',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '5VELmBwJTk-urTuhZdTgew',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

**for my computer , need the following enviroment to run this script**
<br>
/c/Users/junec/anaconda3/python
<br>
/c/Users/junec/anaconda3/Scripts/pip

In [ ]:
!which python
!which pip

In [5]:
#load files
# filepath="data/DBpedia_map_type_entities.json"
# with open(filepath, 'r', encoding='utf-8') as file:
#     DBpedia_map_type_entities = json.load(file)
    
# filepath="data/training_types.json"
# with open(filepath,encoding='utf-8') as json_file:
#     training_map_type_questions = json.load(json_file)

# filepath="data/ElasticSearch_map_type_docID.json"
# with open(filepath, 'r',encoding='utf-8') as f:
#     docID_DBOtype_dict = json.load(f)

filepath="../smart-dataset/datasets/DBpedia/smarttask_dbpedia_train.json"
with open(filepath, 'r') as f:
    smarttask_dbpedia_train = json.load(f)

filepath="data/Knowledgebase_map_type_features_13to16.json"
with open(filepath, 'r',encoding='utf-8') as f:
    map_type_featureVectors13to16 = json.load(f)
    
    

# typeobj=TypeTaxonomy("data/dbpedia_types.tsv")

In [3]:
def extract_features_knowlegebase(
    dp_type: str,
    map_type_featureVectors13to16:Dict
) -> List[float]:
    """Extracts features of a query for knowledge base features.

        Args:
            dp_type: DBO type.
            map_type_featureVectors13to16: dictionary for type and featureVectors13to16.

        Returns:
            List of extracted feature values in a fixed order.
    """
    return map_type_featureVectors13to16[dp_type]

In [21]:
# dp_type="dbo:MusicFestival"
# extract_features_knowlegebase(
#     dp_type,
#     map_type_featureVectors13to16
# ) 

In [20]:
#with first path filter
def prepare_training_data_knowlegebase( smarttask_dbpedia_train:List[Dict],
                            map_type_featureVectors13to16:Dict[str,List[float]],
                            es: Elasticsearch
                            )-> Tuple[List[List[float]], List[int]]:
    """X and y label for knowledge base features. Save to csv file.
        Args:
            smarttask_dbpedia_train: smarttask training set.
            map_type_featureVectors13to16: dictionary for type and featureVectors13to16.
            es: Elasticsearch object instance.

    """
    
    count=0
    j=0
    with open("data/for_training_knowlegebase.csv", 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)


        for entry in smarttask_dbpedia_train:
            if j%100==0:
                print("--------------------j:",j)

            j+=1
            if entry['question']==None:
                continue

            question_processed=preprocess(entry['question'])

            if entry['category']=='resource':
                if count%50==0:
                    print("--------------------count:",count)
                    print("-----------------",datetime.datetime.now())
                count+=1
                print("question_processed:",question_processed)
                for DBOtype in entry['type']: 
                    try:
                        features=extract_features_knowlegebase(dp_type,
                                                                map_type_featureVectors13to16) 
                    except BaseException as err:
                        print("------------error for type:",DBOtype,entry['question'])
                        print(f"Unexpected {err}, {type(err)}")  
                        raise

                    writer.writerow(features)
                    writer.writerow([1])

                #deal with top 30 documents
                hits = es.search(
                    index="dbpdiea_type_centric", q=question_processed, _source=True, size=30
                )["hits"]["hits"]
                rank_list= [hit['_source']["type"] for hit in hits]

                for DBOtype in rank_list:
                    if DBOtype not in entry['type']:
                        try:
                            features=extract_features_knowlegebase(dp_type,
                                                                    map_type_featureVectors13to16)
                        except BaseException as err:
                            print("------------error for type:",DBOtype,entry['question'])
                            print(f"Unexpected {err}, {type(err)}")  
                            raise
                        #print(features)

                        writer.writerow(features)
                        writer.writerow([0])

    print(f'{count} features has been saved,{j} questions have been processed')

    return None




In [ ]:
prepare_training_data_knowlegebase(smarttask_dbpedia_train,
                                        map_type_featureVectors13to16,
                                        es) 